In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from dataclasses import dataclass

 Mainly used for classes containing data

In [7]:
@dataclass  # decoratro to turn class into a dataclass
class DataClassCard:
    rank: str  # fields
    suit: str

In [8]:
queen_of_hearts = DataClassCard('Q', 'Hearts')
queen_of_hearts.rank

'Q'

In [10]:
# compare to regular class : in dataclass don't have to write init to simply pass the fields to self

class RegularCard:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

In [12]:
queen_of_hearts  # nice default __repr__ method

DataClassCard(rank='Q', suit='Hearts')

Such structs with data fields could be replaced by dicts or namedtuples, but it's preferred now to use dataclasses, and they provide more support and have many more features

(e.g. namedtuple is immutable, in dicts there's no dot-access, etc etc)

In [14]:
from dataclasses import make_dataclass

# simplified dataclass creation (similar to namedtuple)
Position = make_dataclass('Position', ['name', 'lat', 'lon'])

In [15]:
# __init__, __repr__, __eq__ have default implementations in a dataclass

In [21]:
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float = 0.0  # fields default values, can also use typehints
    lat: float = 0.0

pos = Position('Greenwich', lat=51.8)
pos

pos.lat
pos.lat = -40  # setitem will also work
pos

Position(name='Greenwich', lon=0.0, lat=51.8)

51.8

Position(name='Greenwich', lon=0.0, lat=-40)

In [19]:
p.lon =10
p

Position(name='Greenwich', lon=10, lat=51.8)

In [22]:
from math import asin, cos, radians, sin, sqrt

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

    def distance_to(self, other):  # can add other methods as usual
        r = 6371  
        lam_1, lam_2 = radians(self.lon), radians(other.lon)
        phi_1, phi_2 = radians(self.lat), radians(other.lat)
        h = (sin((phi_2 - phi_1) / 2)**2
             + cos(phi_1) * cos(phi_2) * sin((lam_2 - lam_1) / 2)**2)
        return 2 * r * asin(sqrt(h))

In [26]:
from dataclasses import dataclass
from typing import List

@dataclass
class PlayingCard:
    rank: str
    suit: str

@dataclass
class Deck:
    cards: List[PlayingCard]  # creating list of objects via dataclass


queen_of_hearts = PlayingCard('Q', 'Hearts')
ace_of_spades = PlayingCard('A', 'Spades')
two_cards = Deck([queen_of_hearts, ace_of_spades])
two_cards
two_cards.cards[0]  # access to the cards

Deck(cards=[PlayingCard(rank='Q', suit='Hearts'), PlayingCard(rank='A', suit='Spades')])

PlayingCard(rank='Q', suit='Hearts')

In [36]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

def make_french_deck():
    return [PlayingCard(r, s) for s in SUITS for r in RANKS]

make_french_deck()[:5]


# let's say we want make_french_deck to be a default generator or theh cards field
# this is done using dataclasses.field 
from dataclasses import field

@dataclass
class Deck:
    cards: List[PlayingCard] = field(default_factory=make_french_deck)  # passing make_french_deck as default factory
    id: int = field(default=4)  # field accepts default= arg for a single default values
Deck()

# for a single default value could write id: int = 4 : but field() has other args :
# repr=False will hide it from print, compare=False will exclude it from using in comparisons etc
# init=False indicates that the field will not be passed in the constructor (will be calculated later using other fields)

# can add __post_init__ method : thing that will be executed after the regular automatic __init__ (e.g. to calculate some derived fields)

[PlayingCard(rank='2', suit='♣'),
 PlayingCard(rank='3', suit='♣'),
 PlayingCard(rank='4', suit='♣'),
 PlayingCard(rank='5', suit='♣'),
 PlayingCard(rank='6', suit='♣')]

Deck(cards=[PlayingCard(rank='2', suit='♣'), PlayingCard(rank='3', suit='♣'), PlayingCard(rank='4', suit='♣'), PlayingCard(rank='5', suit='♣'), PlayingCard(rank='6', suit='♣'), PlayingCard(rank='7', suit='♣'), PlayingCard(rank='8', suit='♣'), PlayingCard(rank='9', suit='♣'), PlayingCard(rank='10', suit='♣'), PlayingCard(rank='J', suit='♣'), PlayingCard(rank='Q', suit='♣'), PlayingCard(rank='K', suit='♣'), PlayingCard(rank='A', suit='♣'), PlayingCard(rank='2', suit='♢'), PlayingCard(rank='3', suit='♢'), PlayingCard(rank='4', suit='♢'), PlayingCard(rank='5', suit='♢'), PlayingCard(rank='6', suit='♢'), PlayingCard(rank='7', suit='♢'), PlayingCard(rank='8', suit='♢'), PlayingCard(rank='9', suit='♢'), PlayingCard(rank='10', suit='♢'), PlayingCard(rank='J', suit='♢'), PlayingCard(rank='Q', suit='♢'), PlayingCard(rank='K', suit='♢'), PlayingCard(rank='A', suit='♢'), PlayingCard(rank='2', suit='♡'), PlayingCard(rank='3', suit='♡'), PlayingCard(rank='4', suit='♡'), PlayingCard(rank='5', suit='♡

@dataclass decorator has some useful args
* frozen=True will make it immutable
* order=True will create ordering
* automatical ordering will compare fields in lexicographic order which may be the correct thing for us:
    - a work around may be to add sort_index field (as the first field in the dataclass) with init=False, repr=False 
        and that is calculated in the __post_init__ using the existing fields and defines the right order
    - see an example below:

In [38]:
# creting custom index for ordering (must be the first field, but should not be in init or repr)
@dataclass(order=True)
class PlayingCard:
    sort_index: int = field(init=False, repr=False)
    rank: str
    suit: str

    def __post_init__(self):
        self.sort_index = (RANKS.index(self.rank) * len(SUITS)
                           + SUITS.index(self.suit))

In [40]:
# dataclass inheritance

@dataclass
class Position:
    name: str
    lon: float
    lat: float

@dataclass
class Capital(Position):
    country: str  # adding extra field w.r.t the base class

Capital('London', 53, 0, 'UK')  # constructor waits for both base and child fields

Capital(name='London', lon=53, lat=0, country='UK')

In [46]:
@dataclass
class Position:
    name: str
    lon: float
    lat: float = field(default=0)  # if a default value is used in the base class, all fields in a subclass must have default values 

@dataclass
class Capital(Position):
    # country: str  # won't work, need to make it kwarg
    country: str = field(default=None)  # now all good, can use country key 

Capital('London', 53, country='UK')    # skipping lat (use default), but defining country

Capital(name='London', lon=53, lat=0, country='UK')